In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adagrad

import os
import numpy as np

C:\Users\PC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Pre-defined parameters
batch_size = 128
num_classes = 10
epochs = 100
subtract_pixel_mean = True

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'alexnet_tubule_segmentation_baseline_model.h5'

In [3]:
## Load Dataset
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [40]:
def build_alexnet_cnn(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32,(3,3),activation="relu", padding="same")(input_layer)
    x = MaxPooling2D(pool_size=(3,3),strides=2)(x)
    
    x = Conv2D(32,(5,5),activation="relu",padding="same")(x)
    x = AveragePooling2D(pool_size=(3,3),strides=2)(x)
    
    x = Conv2D(64,(5,5),activation="relu",padding="same")(x)
    x = AveragePooling2D(pool_size=(3,3),strides=2)(x)

        
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    
    x = Dropout(0.3)(x)
    output_layer = Dense(10, activation="softmax")(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [41]:
# initiate RMSprop optimizer
opt = keras.optimizers.Adagrad(lr=0.001, decay=1e-3*4)

In [42]:
## https://keras.io/examples/cifar10_resnet/

In [43]:
# Let's train the model using Adagrad optimizer
model = build_alexnet_cnn(input_shape=x_train.shape[1:])
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 15, 15, 32)        25632     
_________________________________________________________________
average_pooling2d_9 (Average (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 7, 7, 64)          51264     
_________________________________________________________________
average_pooling2d_10 (Averag (None, 3, 3, 64)          0         
__________

In [45]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 8s 163us/step - loss: 2.0200 - acc: 0.2449 - val_loss: 1.8754 - val_acc: 0.3285
Epoch 2/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.9328 - acc: 0.2773 - val_loss: 1.8345 - val_acc: 0.3447
Epoch 3/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.9084 - acc: 0.2898 - val_loss: 1.8156 - val_acc: 0.3531
Epoch 4/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.8978 - acc: 0.2900 - val_loss: 1.8044 - val_acc: 0.3551
Epoch 5/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.8883 - acc: 0.2972 - val_loss: 1.7970 - val_acc: 0.3589
Epoch 6/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.8777 - acc: 0.3001 - val_loss: 1.7905 - val_acc: 0.3606
Epoch 7/100
50000/50000 [==============================] - 4s 74us/step - loss: 1.8718 - acc: 0.3032 - val_loss: 1.7857 -

KeyboardInterrupt: 